In [1]:
import os
import pickle as pkl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.manifold import TSNE
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
DATASET_ROOT_PATH = '/home/ubuntu/projects/ddos/cicddos_data/core'
X_TRAIN_PATH = os.path.join(DATASET_ROOT_PATH, 'X_train.csv')
Y_TRAIN_PATH = os.path.join(DATASET_ROOT_PATH, 'Y_train.csv')
X_TEST_PATH = os.path.join(DATASET_ROOT_PATH, 'X_test.csv')
Y_TEST_PATH = os.path.join(DATASET_ROOT_PATH, 'Y_test.csv')

In [3]:
X_train = pd.read_csv(X_TRAIN_PATH, low_memory=False)
Y_train = pd.read_csv(Y_TRAIN_PATH, low_memory=False)

X_test = pd.read_csv(X_TEST_PATH, low_memory=False)
Y_test = pd.read_csv(Y_TEST_PATH, low_memory=False)

In [4]:
label_cols = ['Label']

numeric_cols = ['URG Flag Count', 'Fwd PSH Flags',
    'RST Flag Count', 'CWE Flag Count', 'Packet Length Std',
    'Init_Win_bytes_forward', 'Fwd Packet Length Std', 'Active Mean',
    'Idle Mean', 'Avg Fwd Segment Size',
    'Fwd Packet Length Mean', 'Fwd Packets/s', 'Packet Length Mean',
    'Average Packet Size', 'Fwd IAT Min', 'Flow Duration',
    'Fwd IAT Total', 'Active Std', 'SYN Flag Count', 'Fwd IAT Std', 
    'Flow IAT Std', 'Subflow Fwd Bytes', 'Total Length of Fwd Packets']

categoric_cols = [
    'Protocol',
    'Inbound'
]

## Custom Transformers

In [5]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.col_names].values

# Pipeline

#### individual pipelines

In [6]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(numeric_cols)),
    ('std_scaler', StandardScaler())
])

In [7]:
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(categoric_cols)),
    ('one_hot_encoder', OneHotEncoder(sparse=False))
])

#### Feature Union

In [8]:
full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline)
])

# Run pipeline

In [9]:
X = full_pipeline.fit_transform(X_train)

# Train

In [ ]:
from sklearn import svm

In [12]:
svc = svm.SVC()

In [ ]:
svc.fit(X, Y_train.values.reshape(Y_train.values.shape[0], ))

# Validation

#### calculating error

In [ ]:
scores = cross_val_score(rf_clf, full_pipeline.fit_transform(X_test), Y_test.values.reshape(Y_test.shape[0], ), 
                         scoring='neg_mean_squared_error', cv=10)

In [ ]:
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
tree_rmse_scores

#### Calculating accuracy

In [ ]:
accuracies = cross_val_score(svc, full_pipeline.fit_transform(X_test), Y_test.values.reshape(X_test.shape[0], ), cv=10, scoring='accuracy')
print("Model Accuracy:")
for i in accuracies:
    print(i)